In [2]:
import os
os.chdir('/workspaces/Kidney-Disease-Classification-DL')

In [3]:
%pwd

'/workspaces/Kidney-Disease-Classification-DL'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_url : str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories



In [6]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            
            create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
             root_dir=config.root_dir,
             source_url=config.source_url,
             local_data_file=config.local_data_file,
             unzip_dir=config.unzip_dir
        )
        return data_ingestion_config
     

In [7]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config
    
    def download_file(self)->str:

        try:
            dataset_url = self.config.source_url
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion",exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id, zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")
        except Exception as e:
            raise e
    
    def extract_zip_files(self):

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_files()
except Exception as e:
    raise e

[2026-02-04 10:00:19,032: INFO: common: yaml file: config/config.yaml loaded successfully]
[2026-02-04 10:00:19,047: INFO: common: yaml file: params.yaml loaded successfully]
[2026-02-04 10:00:19,049: INFO: common: created directory at: artifacts]
[2026-02-04 10:00:19,051: INFO: common: created directory at: artifacts/data_ingestion]
[2026-02-04 10:00:19,051: INFO: 1254834476: Downloading data from https://drive.google.com/file/d/1iCyh_bNa2wbuU330HfkPsIXU8Yy8oarp/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1iCyh_bNa2wbuU330HfkPsIXU8Yy8oarp
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1iCyh_bNa2wbuU330HfkPsIXU8Yy8oarp&confirm=t&uuid=c120c7a4-9ac3-45ed-b6bd-02e6d04408c9
To: /workspaces/Kidney-Disease-Classification-DL/artifacts/data_ingestion/data.zip
100%|██████████| 57.7M/57.7M [00:01<00:00, 44.8MB/s]

[2026-02-04 10:00:22,879: INFO: 1254834476: Downloaded data from https://drive.google.com/file/d/1iCyh_bNa2wbuU330HfkPsIXU8Yy8oarp/view?usp=sharing into file artifacts/data_ingestion/data.zip]
